<a href="https://colab.research.google.com/github/akashwaf/finvasia01/blob/main/simple_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#### parameters that can be changed #############3 

initial_premium =  100
new_position_multiplier = 1.10
sl_multiplier = 1.10
max_sl_multiplier =  1.21

lot_size = 25




expiry = '22-DEC-2022'
selected_instrument = 'BANKNIFTY'



premium = initial_premium 

sl = premium* sl_multiplier
max_sl = premium* max_sl_multiplier 




In [6]:
# run to install library temporarily 

!git clone https://github.com/Shoonya-Dev/ShoonyaApi-py.git

# this are the libraries mentioned in requirements.txt
!pip install requests
!pip install websocket_client
!pip install /content/ShoonyaApi-py/dist/NorenRestApiPy-0.0.22-py2.py3-none-any.whl
!pip install pyyaml


!pip install pyotp




# !pip install talib-binary             # recently stopped working 
# !pip install TA-Lib-binary               # working now 



fatal: destination path 'ShoonyaApi-py' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./ShoonyaApi-py/dist/NorenRestApiPy-0.0.22-py2.py3-none-any.whl
NorenRestApiPy is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:

import multiprocessing as mp 
import time
import math
import os , sys
import logging
import pandas as pd 
import pprint
import time
import yaml
import warnings 

# manually installed imports 
import pyotp


import os, time
os.environ['TZ'] = "Asia/Calcutta" # set new timezone
time.tzset()

In [8]:


sold_ce = 1
sold_pe = 2

sold_ltp =  {
    
    sold_ce: 0000,
    sold_pe: 0000
  
}






# making global variable and using it in other functions
token_df = pd.read_csv('https://shoonya.finvasia.com/NFO_symbols.txt.zip')

token_df  = token_df [token_df['Symbol'] == selected_instrument]  
token_df  = token_df [token_df['Expiry'] == expiry]
token_df.sort_values(by =  'StrikePrice', inplace = True)
global_token_df = token_df 


ce_token_df  = token_df [token_df['OptionType'] == 'CE']
ce_token_list  = ce_token_df[ 'Token'].tolist()
ce_token_str_list = [str(x) for x in ce_token_list]


pe_token_df  = token_df [token_df['OptionType'] == 'PE']
pe_token_list  = pe_token_df['Token'].tolist()
pe_token_str_list = [str(x) for x in pe_token_list]



ce_lp_data = {}
pe_lp_data = {}
lp_data ={}


if True :
    if 'feed_jason' not in globals():    
      feed_opened = False
    if 'subscribed' not in globals():
      subscribed = []

  


In [9]:

if True: 
  # warnings.simplefilter("ignore")
  feed_opened = False
  feedJson = {}
  socket_opened = False
  orderJson = {}


  acknowledgement_data = {}
  lp_data = {}
  ce_lp_data = {}
  pe_lp_data = {}



  def event_handler_feed_update(tick_data):
      
      global data_response_type
      global ce_lp_data
      global pe_lp_data
      global lp_data 
      global sold_ltp
      global sold_ce 
      global sold_pe





      # UPDATE LIVE LTP IN  GLOBAL JASON FORMAT VARIABLE 
      if 'lp' in tick_data:
        lp_data[tick_data['tk']] =   tick_data['lp']


        if tick_data['tk'] in ce_token_str_list :
          ce_lp_data [tick_data['tk']] = tick_data['lp']
        
        
        if tick_data['tk'] in pe_token_str_list :
          pe_lp_data [tick_data['tk']] = tick_data['lp']

             
        
        if tick_data['tk'] == sold_ce or tick_data['tk'] == sold_pe : 

          if tick_data['tk'] == sold_ce: 
            sold_ltp[sold_ce]  =  tick_data['lp']

            
          if tick_data['tk'] == sold_pe: 
            sold_ltp[sold_pe]  =  tick_data['lp']



  def event_handler_order_update(order_data):
      print('orderupdate')
      
      print(order_data)

  def open_callback():
      global feed_opened
      feed_opened = True


  def close_callback():
      global feed_opened
      feed_opened = False

  def setupWebSocket() :
    global fee_opened
    if feed_opened == True:
      api.close_websocket()

    
    api.start_websocket( order_update_callback=event_handler_order_update,
                      subscribe_callback=event_handler_feed_update, 
                      socket_open_callback= open_callback,
                        socket_close_callback = close_callback)

    time.sleep(1)
    
    while(feed_opened==False):
        # print('here only')
        pass


    return True 



In [10]:
if True : 
  if True:
    from NorenRestApiPy.NorenApi import  NorenApi
    from threading import Timer
    import pandas as pd
    import time
    import concurrent.futures

    api = None
    class Order:
        def __init__(self, buy_or_sell:str = None, product_type:str = None,
                    exchange: str = None, tradingsymbol:str =None, 
                    price_type: str = None, quantity: int = None, 
                    price: float = None,trigger_price:float = None, discloseqty: int = 0,
                    retention:str = 'DAY', remarks: str = "tag",
                    order_id:str = None):
            self.buy_or_sell=buy_or_sell
            self.product_type=product_type
            self.exchange=exchange
            self.tradingsymbol=tradingsymbol
            self.quantity=quantity
            self.discloseqty=discloseqty
            self.price_type=price_type
            self.price=price
            self.trigger_price=trigger_price
            self.retention=retention
            self.remarks=remarks
            self.order_id=None


        #print(ret)

        


    def get_time(time_string):
        data = time.strptime(time_string,'%d-%m-%Y %H:%M:%S')

        return time.mktime(data)


    class ShoonyaApiPy(NorenApi):
        def __init__(self):
            NorenApi.__init__(self, host='https://api.shoonya.com/NorenWClientTP/', websocket='wss://api.shoonya.com/NorenWSTP/')        
            global api
            api = self

        def place_basket(self, orders):

            resp_err = 0
            resp_ok  = 0
            result   = []
            with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:

                future_to_url = {executor.submit(self.place_order, order): order for order in  orders}
                for future in concurrent.futures.as_completed(future_to_url):
                    url = future_to_url[future]
                try:
                    result.append(future.result())
                except Exception as exc:
                    print(exc)
                    resp_err = resp_err + 1
                else:
                    resp_ok = resp_ok + 1

            return result
                    
        def placeOrder(self,order: Order):
            ret = NorenApi.place_order(self, buy_or_sell=order.buy_or_sell, product_type=order.product_type,
                                exchange=order.exchange, tradingsymbol=order.tradingsymbol, 
                                quantity=order.quantity, discloseqty=order.discloseqty, price_type=order.price_type, 
                                price=order.price, trigger_price=order.trigger_price,
                                retention=order.retention, remarks=order.remarks)
            #print(ret)

            return ret




  def do_login():
    #enable dbug to see request and responses
    logging.basicConfig(level=logging.DEBUG)

    #start of our program
    api = ShoonyaApiPy()

    #credentials
    user        = 'FA13620'
    pwd       =  '2012Ak@sh'
    token =     "52R5K462BA4H5TJ7SS746S5534364527"
    factor2     =  pyotp.TOTP(token).now()
    vc          = 'FA13620_U'
    app_key     = '9088c808ac3522b35705a4acd8a46d03'
    imei        = 'abc1234'




    ret = api.login(userid=user, password=pwd, twoFA=factor2, vendor_code=vc, api_secret=app_key, imei=imei)


    print(factor2)

    print(ret)



  def get_token_df (opt_type= 'CE', expiry = '08-DEC-2022'):

    # read
    global selected_instrument
        
    token_df = pd.read_csv('https://shoonya.finvasia.com/NFO_symbols.txt.zip')
    
    
    filtered_token  = token_df [token_df['Symbol'] == selected_instrument]  
    filtered_token  = filtered_token [filtered_token['Expiry'] == expiry]
    filtered_token  = filtered_token [filtered_token['OptionType'] == opt_type]
    print(filtered_token)

    filtered_token.sort_values(by =  'StrikePrice', inplace = True)


    return filtered_token












  def make_ready_to_subscribe( token_df  ):

    print(token_df)

    token_list = token_df['Token'].tolist()

    
    # token_list = token_df[80:len(token_df)]['Token'].tolist()

    ready_to_subscribe = []
    for i in token_list:

      ready_to_subscribe.append( 'NFO|'+str(i) )


    print(ready_to_subscribe)


    return ready_to_subscribe







  def select_opt_by_premium (premium  = 100):
    # returns token and price of the selected premium 
    # for that it looks into ce_lp_data which is updated by the websocket 

    diff  = 1000000
    nearest_price = 1000000
    nearest_token = 9000000000

    for x, y  in ce_lp_data.items():


      if abs(float(y) - premium) < diff :

        diff = abs(float(y) - premium)
        nearest_token = x
        nearest_price = float(y)


    
      # print(x , y )

    print('nearest token , nearest_price, and difference is ')
    print(nearest_token,nearest_price , diff)


    return nearest_token , nearest_price
















In [11]:

def select_opt_by_premium_type (premium , opt_type  ):
  # returns token and price of the selected premium 
  # for that it looks into ce_lp_data which is updated by the websocket 

  diff  = 1000000
  nearest_price = 1000000
  nearest_token = 9000000000

  global ce_lp_data
  global pe_lp_data
  
  if opt_type == 'CE':

    for x, y  in ce_lp_data.items():

      if abs(float(y) - premium) < diff :

        diff = abs(float(y) - premium)
        nearest_token = x
        nearest_price = float(y)
        # print(x , y )

    print('nearest token , nearest_price, and difference is ')
    print(nearest_token,nearest_price , diff)

    return nearest_token , nearest_price




  if opt_type == 'PE' :
        
    for x, y  in pe_lp_data.items():

        if abs(float(y) - premium) < diff :

          diff = abs(float(y) - premium)
          nearest_token = x
          nearest_price = float(y)

          # print(x , y )

    print('nearest token , nearest_price, and difference is ')
    print(nearest_token,nearest_price , diff)


    return nearest_token , nearest_price

  
  return 0 , 0


def exit_position( exit_token  ):

  exit_tradingsymbol = token_to_tradingsymbol ( int(exit_token)  )

  

  ret = api.place_order(buy_or_sell='B',
                        product_type='I',
                        exchange='NFO',
                        tradingsymbol= exit_tradingsymbol,
                        quantity=lot_size* 1,
                        discloseqty=0, 
                        price_type='MKT', 
                        # price=200.00, 
                        # trigger_price=199.50,
                        retention='DAY',
                        remarks='mkt_order')
  




  # exit position with exit_tradingsymbol
  pass




def make_position( new_premium ,  new_type ):

  new_token, new_price  = select_opt_by_premium_type ( premium =  new_premium ,opt_type  =  new_type)




  print('')
  print('making new_position with new price = ' , new_price)

  new_tradingsymbol = token_to_tradingsymbol ( int(new_token) )
  print('new trading symbol = ', new_tradingsymbol)
  


  time.sleep(5)



  ret = api.place_order(buy_or_sell='S',
                        product_type='I',
                        exchange='NFO',
                        tradingsymbol= new_tradingsymbol,
                        quantity=25,
                        discloseqty=0, 
                        price_type='MKT', 
                        # price=200.00, 
                        # trigger_price=199.50,
                        retention='DAY',
                        remarks='paper_trading')
  
  print(ret)


  # place sell order with new_tradingsymbol ####################################

  
  if True :     # if order place successfully

    
    global sold_ltp 
    global sold_ce
    global sold_pe

    if new_type == 'CE': 
      del sold_ltp [ sold_ce]
      sold_ltp [new_token]  =   0
      sold_ce = new_token


    
    if new_type == 'PE': 
      del sold_ltp [ sold_pe]
      sold_ltp[ new_token] = 0
      sold_pe = new_token



In [12]:
def token_to_tradingsymbol ( token ):



  global global_token_df

  try : 
    trading_symbol = global_token_df [ global_token_df [ 'Token']  == token ]

    trading_symbol = trading_symbol ['TradingSymbol'].iloc[0]
    print(trading_symbol)

    return trading_symbol

  except IndexError as e :

    print('invalid token  ')
    return 0
  
  


token_to_tradingsymbol( int( '43308') )


invalid token  


0

In [13]:
from pandas.io.formats.printing import pprint_thing
#@title
def exit_position( exit_token  ):

  exit_tradingsymbol = token_to_tradingsymbol ( int(exit_token)  )

  print('inside exit position , now trying to exit position ')

  

  ret = api.place_order(buy_or_sell='B',
                        product_type='I',
                        exchange='NFO',
                        tradingsymbol= exit_tradingsymbol,
                        quantity= lot_size *1 ,
                        discloseqty=0, 
                        price_type='MKT', 
                        # price=200.00, 
                        # trigger_price=199.50,
                        retention='DAY',
                        remarks='mkt_order')
  
  print(ret )
  




  # exit position with exit_tradingsymbol
  pass




def make_position( new_premium ,  new_type ):

  new_token, new_price  = select_opt_by_premium_type ( premium =  new_premium ,opt_type  =  new_type)


  print('')
  print('making new_position with new price = ' , new_price)

  new_tradingsymbol = token_to_tradingsymbol ( int(new_token) )
  print('new trading symbol = ', new_tradingsymbol)
  
  print('')
  print('')

  time.sleep(1)



  ret = api.place_order(buy_or_sell='S',
                        product_type='I',
                        exchange='NFO',
                        tradingsymbol= new_tradingsymbol,
                        quantity=25,
                        discloseqty=0, 
                        price_type='MKT', 
                        # price=200.00, 
                        # trigger_price=199.50,
                        retention='DAY',
                        remarks='mkt_order')
  
  print(ret)


  # place sell order with new_tradingsymbol ####################################

  
  if True :     # if order place successfully

    
    global sold_ltp 
    global sold_ce
    global sold_pe

    if new_type == 'CE': 
      del sold_ltp [ sold_ce]
      sold_ltp [new_token]  =   0
      sold_ce = new_token



      




    
    if new_type == 'PE': 
      del sold_ltp [ sold_pe]
      sold_ltp[ new_token] = 0
      sold_pe = new_token




In [14]:

do_login()

global token_df
ready_to_subscribe = make_ready_to_subscribe(token_df)


# it starts websocket 


global feed_opened 

if feed_opened == True :

  api.close_websocket()
  time.sleep(1)
  setupWebSocket()
  feed_opened = True
else:
  
  setupWebSocket()
  feed_opened = True


time.sleep(5)




874859
{'request_time': '10:21:24 16-12-2022', 'actid': 'FA13620', 'uname': 'AKASH RAJESHKUMAR PATEL', 'prarr': [{'prd': 'C', 's_prdt_ali': 'CNC', 'exch': ['NSE', 'BSE', 'NIPO']}, {'prd': 'M', 's_prdt_ali': 'NRML', 'exch': ['NFO', 'CDS', 'BCD', 'MCX']}, {'prd': 'I', 's_prdt_ali': 'MIS', 'exch': ['NSE', 'BSE', 'NFO', 'CDS', 'BCD', 'MCX']}, {'prd': 'H', 's_prdt_ali': 'CO', 'exch': ['NSE', 'NFO', 'CDS', 'MCX', 'BSE']}, {'prd': 'B', 's_prdt_ali': 'BO', 'exch': ['NSE', 'NFO', 'CDS', 'MCX', 'BSE']}], 'stat': 'Ok', 'susertoken': 'bee3f351193fcfe3d6f2d01c0174e8400bc54e65ebf42271299355f33f882998', 'email': 'AKSPMONEY1@GMAIL.COM', 'uid': 'FA13620', 'brnchid': 'HO', 'orarr': ['LMT', 'MKT', 'SL-LMT', 'SL-MKT'], 'exarr': ['NSE', 'NFO', 'CDS', 'MCX', 'BSE', 'BCD', 'NIPO'], 'values': ['1'], 'mws': {'1': []}, 'brkname': 'FINV', 'lastaccesstime': '1671166284'}
      Exchange  Token  LotSize     Symbol           TradingSymbol  \
75675      NFO  51251       25  BANKNIFTY  BANKNIFTY22DEC22C32500   
75669 

In [15]:


# api.unsubscribe(ready_to_subscribe)

api.subscribe(ready_to_subscribe, feed_type='t')






In [16]:
# api.close_websocket()
# feed_opened = False

In [17]:

time.sleep(3)
if sold_ce == 1 and sold_pe  == 2  : 

   
  make_position ( new_premium =  premium  ,  new_type = 'CE' )

  time.sleep(5)
  
  make_position ( new_premium =  premium  ,  new_type = 'PE' )


  pass


while True:

  if float(sold_ltp [sold_ce])  > sl  or  float (sold_ltp[sold_pe]  ) > sl: 
    

    if sl > max_sl : 
      pass
      # exit all position and stop trading

      exit_position( sold_ce)
      exit_position (sold_pe) 

      sys.exit(0)




    if float(sold_ltp[sold_ce]) > sl : 
      pass

      exit_position(  sold_pe   )

      make_position ( new_premium =  premium *  new_position_multiplier  ,  new_type = 'PE' )

      # global premium ,sl
      premium = new_position_multiplier * premium 
      sl = premium * sl_multiplier

      
        # it will update sold_ce , sl , premium , sold_ltp 

    if float(sold_ltp[sold_pe]) > sl : 
      pass
      
      exit_position(  sold_ce   )
      
      premium 
      make_position ( new_premium =  premium  * new_position_multiplier  ,  new_type = 'CE' )
      
      # global premium ,sl
      premium = new_position_multiplier * premium 
      sl = premium * sl_multiplier




nearest token , nearest_price, and difference is 
51468 103.55 3.549999999999997

making new_position with new price =  103.55
BANKNIFTY22DEC22C43700
new trading symbol =  BANKNIFTY22DEC22C43700


{'request_time': '10:21:35 16-12-2022', 'stat': 'Ok', 'norenordno': '22121600172301'}
orderupdate
{'t': 'om', 'norenordno': '22121600172301', 'uid': 'FA13620', 'actid': 'FA13620', 'exch': 'NFO', 'tsym': 'BANKNIFTY22DEC22C43700', 'trantype': 'S', 'qty': '25', 'prc': '0.00', 'pcode': 'I', 'remarks': 'mkt_order', 'status': 'PENDING', 'reporttype': 'NewAck', 'prctyp': 'MKT', 'ret': 'DAY', 'exchordid': '', 'dscqty': '0'}
orderupdate
{'t': 'om', 'norenordno': '22121600172301', 'uid': 'FA13620', 'actid': 'FA13620', 'exch': 'NFO', 'tsym': 'BANKNIFTY22DEC22C43700', 'trantype': 'S', 'qty': '25', 'prc': '0.00', 'pcode': 'I', 'remarks': 'mkt_order', 'rejreason': 'RED:Margin Shortfall:INR 1,42,216.73 Available:INR 0.00 for C-FA13620 [SHOONYA]', 'status': 'REJECTED', 'reporttype': 'Rejected', 'prctyp': 'MK

KeyboardInterrupt: ignored

In [ ]:
# setupWebSocket()

In [ ]:
# import datetime as dt 
# import pytz


# dt1 = dt.datetime.now()
# print(dt1 )


# dt2 = dt.datetime.now( pytz.timezone( "Asia/Calcutta" ) ).time()
# print(dt2)




# dt3 = dt.datetime.local_time()


# print(dt3)

In [ ]:
# pprint.pprint(api.get_order_book())




# order_book  = api.get_order_book()

# if order_book[0]['status']  == 'REJECTED' :
#   print( ' rejected order is this ')

In [ ]:
print( ce_lp_data)

print(pe_lp_data)


print( lp_data)


print(ready_to_subscribe)

In [ ]:


# print(sold_ltp [sold_ce])

print(token_to_tradingsymbol( int( sold_ce)))




print(token_to_tradingsymbol( int( sold_pe)))

# print(type(float(sold_ltp [sold_ce])))





In [ ]:
# s1 = [1 ,2 , 3 , 4, 5,15.645,65,666,565]
# s2 = [1 ,2 , 3 , 5, 25 , 5 , 54,47]

# subsribed = ready_to_subscribe 
 
# def add_subs () : 
#   for i in ready_to_subsribe: 

#     if i not in subsribed: 
#       subsribed.append(i)


#   print(s2)






# def remove_subs () : 
#   for i in ready_to_subsribe: 

#     if i in subsribed: 
#       print('this is not present  ',i )
#       subsribed.remove(i)


#   print(s2)

# remove_subs() 